In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
from pathlib import Path
import math
import swifter
from sklearn import metrics
from statsmodels.sandbox.stats.runs import mcnemar
plt.style.reload_library()
plt.style.use(['science'])

In [ ]:
!ls -lah analysis/not_uploaded/IA2NIMA/AVA/

In [ ]:
df_l = []
for p in Path("analysis/not_uploaded/IA2NIMA/AVA/").iterdir():
    tmp = pd.read_csv(p)
    tmp["scores"] = tmp["scores"].swifter.apply(eval)
    tmp["score"] = tmp["scores"].swifter.apply(lambda row: sum([row[i] * (i+1) for i in range(len(row))]))
    tmp["img"] = tmp["img"].swifter.apply(lambda row: row.split(".")[0])
    tmp["quality"] = tmp["score"].apply(lambda row: 1 if row > 5 else 0)
    tmp["quality"] = tmp["quality"].astype(int)
    tmp["img"] = tmp["img"].astype(int)

    tmp.drop(columns=["scores"], inplace=True)
    tmp = tmp.rename(columns={"score":"score_" + p.stem.split("AVA.")[1], "quality": "quality_" + p.stem.split("AVA.")[1]})

    tmp = tmp.set_index("img")
    df_l.append(tmp)

df = df_l[0].join(df_l[1:])

In [ ]:
gt = pd.read_csv("analysis/not_uploaded/AVA_gt.txt", sep=" ").drop(columns=["Unnamed: 0", "semanticTagID1", "semanticTagID2", "challengeID"])

In [ ]:
gt["votes"] = gt.apply(lambda row: sum(list(row)[1:]), axis=1)
gt["gt_score"] = gt.apply(lambda row: sum([val * (i + 1) for i, val in enumerate(list(row)[1:-1])]), axis=1)
gt["gt_score"] = gt.apply(lambda row: row.gt_score / row.votes, axis=1)
gt["gt_quality"] = gt["gt_score"].apply(lambda row: 1 if row > 5 else 0)
gt["img"] = gt["img"].astype(int)

gt = gt[["img","gt_score", "gt_quality"]]
gt = gt.set_index("img")

In [ ]:
df = gt.join(df).dropna().drop_duplicates()
df

In [ ]:
sns.scatterplot(data=df.rename(columns={"score_one.change_regress.epoch-77.pth":"score"}), x="gt_score", y="score")

In [ ]:
sns.histplot(x=df.rename(columns={"score_one.change_regress.epoch-77.pth":"score"})["score"], ax=plt.gcf().gca(), color="red", binwidth=0.2, alpha=0.3)
sns.histplot(x=df.rename(columns={"gt_score":"ground truth"})["ground truth"], ax=plt.gcf().gca(), color="blue", binwidth=0.2, alpha=0.3)
plt.xlim(1,10)
plt.xticks(range(1,11))

In [ ]:
for score in sorted(df.columns):
    if "gt" in score or not "score" in score:
        continue
    if "epoch-0" in score:
        print()
        print(score)
        print("LCC :", stats.pearsonr(df["gt_score"], df[score])[0])
        print("SRCC:", stats.spearmanr(df["gt_score"], df[score])[0])
        print("ACC :", metrics.accuracy_score(df["gt_quality"], df[score.replace("score", "quality")]))
        print("F1  :", metrics.f1_score(df["gt_quality"], df[score.replace("score", "quality")]))

In [ ]:
for score in sorted(df.columns):
    if "gt" in score or not "score" in score:
        continue
    if "epoch-149" in score:
        print()
        print(score)
        print("LCC :", stats.pearsonr(df["gt_score"], df[score])[0])
        print("SRCC:", stats.spearmanr(df["gt_score"], df[score])[0])
        print("ACC :", metrics.accuracy_score(df["gt_quality"], df[score.replace("score", "quality")]))
        print("F1  :", metrics.f1_score(df["gt_quality"], df[score.replace("score", "quality")]))

In [ ]:
for score in sorted(df.columns):
    if "gt" in score or not "score" in score:
        continue
    if "epoch-0" in score or "epoch-149" in score:
        continue 
    print()
    print(score)
    print("LCC :", stats.pearsonr(df["gt_score"], df[score])[0])
    print("SRCC:", stats.spearmanr(df["gt_score"], df[score])[0])

    print("ACC :", metrics.accuracy_score(df["gt_quality"], df[score.replace("score", "quality")]))
    print("F1  :", metrics.f1_score(df["gt_quality"], df[score.replace("score", "quality")]))

In [ ]:
from scipy.stats import wilcoxon

In [ ]:
signific_df = df[["gt_score", "score_one.change_regress.epoch-77.pth", "score_imagenet.epoch-149.pth"]].rename(columns={"score_one.change_regress.epoch-77.pth": "ours", "score_imagenet.epoch-149.pth":"imagenet"})
signific_df

In [ ]:
wilcoxon(signific_df["ours"]-signific_df["gt_score"], signific_df["imagenet"]-signific_df["gt_score"])

In [ ]:
wilcoxon(signific_df["ours"]-signific_df["gt_score"], signific_df["imagenet"]-signific_df["gt_score"], alternative="less")

In [ ]:
wilcoxon(signific_df["ours"]-signific_df["gt_score"], signific_df["imagenet"]-signific_df["gt_score"], alternative="greater")